# Flask 2

Для того, чтобы удобно было работать с базами в обычных программах (особенно в приложениях), придумали **ORM** (Object-Relational Mapping, объектно-реляционное отображение, или преобразование) - это специальный инструмент для перевода объектов в БД в удобную форму в языке программирования (например, в классы питона). Так как мы не изучали классы, можно привести пример pymorphy, где разбор имел разные атрибуты (которые вызывались через точку) и внутри них значения или более сложная структура (.tag имел еще .tag.POS и тд)

Предположим, что мы хотим сделать лингвистическую анкету и у нас есть такая вводная:


- Мы хотим собрать информацию про информанта (гендер, уровень образования и возраст)
- У нас есть два вопроса на оценку предложений
- И нужно куда-то сохранять результаты


Проще всего все положить в БД и потом просто работать с этим файлом.


**User**
- id
- gender
- education
- age

**Questions**
- id
- text

**Answers**
- id (соответсвует id пользователя)
- q1 (ответ на первй вопрос)
- q2 (ответ на второй вопрос)

In [5]:
import sqlite3

db = sqlite3.connect('untitled/test.db')
cur = db.cursor()

In [4]:
cur.execute(
    """CREATE TABLE answers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    q1` INTEGER,
    q2` INTEGER )
    """)

cur.execute(
    """CREATE TABLE questions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    text TEXT
    )""")

cur.execute(
    """CREATE TABLE 
    user ( 
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    gender TEXT,
    education TEXT,
    age INTEGER )""")

db.commit()

### Как подключить базу к приложению?

Мы должны импортировать фласк и использовать еще один специальный модуль flask-sqlalchemy, который позволяет работать с базой. Главное - это прописать путь к базе - там есть часть ```sqlite3:///```- это обозначение для того, что мы работает с таким типом базы, а дальше путь внутри проекта

In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///test.db'
db = SQLAlchemy(app)

### Как передать устройство своей базы?

Нам необходимо объяснить питону, где какие типы данных у нас лежат, для этого нам нужно описать каждую таблицу.

Не будем пока углубляться в классы питона, просто делаем по образцу.

Каждая таблица описывается отделно, с помощью ```__tablename__``` передается ее имя, а названия столбиков совпадают с названиями в нашей таблице. Еще используется специальная "обертка" для столбца, где можно прописать, какой это типа данных и если это первиный ключ, то на это указать

In [ ]:
class User(db.Model):
    __tablename__ = 'user'  # имя таблицы
    id = db.Column(db.Integer, primary_key=True) # имя колонки = специальный тип (тип данных, первичный ключ)
    gender = db.Column(db.Text)
    education = db.Column(db.Text)
    age = db.Column(db.Integer)


class Questions(db.Model):
    __tablename__ = 'questions'
    id = db.Column(db.Integer, primary_key=True)
    text = db.Column(db.Text)


class Answers(db.Model):
    __tablename__ = 'answers'
    id = db.Column(db.Integer, primary_key=True)
    q1 = db.Column(db.Integer)
    q2 = db.Column(db.Integer)

### Как читать из базы (простые вещи)?

Простейшие запросы можно делать прямо к User, Questions, Answers, например, мы хотим в нашей странице с анкетой предложения для оценки брать из базы

Мы должны

1. Получить эти вопросы
2. Передать на страницу
3. Пройти по вопросам и сгенерировать анкету

Создадим путь ```/questions``` и там считаем вопросы из базы и передадим на страницу через ```render_template```

In [ ]:
@app.route('/questions')
def question_page():
    questions = Questions.query.all() # имя_таблицы.query.взять_все()
    return render_template(
        'questions.html',
        questions=questions
    )

А как теперь это использовать, говорили же про атрибуты? Где они?

Предположим, что у нас есть N вопросов (2 в нашем случае) и они одинаково устроены. Мы можем просто сгенерировать места для ответов и они все по очереди будут выводиться.

```{{question.text}}``` - выводит нам поле text из вопроса
```name="q{{ question.id }}"``` - использует id вопроса, чтобы получить имена для элементов например, ```name="q1"``` (чтобы мы могли потом достать ответ именно на первый вопрос)

In [ ]:
{% for question in questions %}
    <div class="row">
        <p class="col-md-6">{{question.text}}</p>
        <table class="col-md-8">
            <tr>
                <td><input class="radio" type="radio" name="q{{ question.id }}" value=5></td>
                <td><input class="radio" type="radio" name="q{{ question.id }}" value=4></td>
                <td><input class="radio" type="radio" name="q{{ question.id }}" value=3></td>
                <td><input class="radio" type="radio" name="q{{ question.id }}" value=2></td>
                <td><input class="radio" type="radio" name="q{{ question.id }}" value=1></td>
            </tr>
            <tr>
                <td>отлично</td>
                <td>хорошо</td>
                <td>норм</td>
                <td>плохо</td>
                <td>ужасно</td>
            </tr>
        </table>
    </div>
{% endfor %}

### Как писать в базу?

Мы указали в форме ```<form action="/process">```, чтобы данные отправлялись на путь process и мы там будем их обрабатывать:


1. Достать все параметры из адреса с GET параметрами (типа ```gender=female&education=hse```) 
2. Записать в базу
3. Сохранить
4. Сказать пользователю, что все ок
5. Если пришел пустой запрос, то отправить проходить анкету

In [ ]:
@app.route('/process', methods=['get'])
def answer_process():
    
    # если пустой запрос, то отправить проходить анкету
    if not request.args:
        return redirect(url_for('question_page'))
    
    # получаем значения ответов
    gender = request.args.get('gender')
    education = request.args.get('education')
    age = request.args.get('age')
    
    # записываем в базу
    user = User(
        age=age,
        gender=gender,
        education=education
    )
    db.session.add(user)
    db.session.commit()
    
    # обновляем user'a, чтобы его ответ записать с таким же id
    db.session.refresh(user)
    
    # это же делаем с ответом
    q1 = request.args.get('q1')
    q2 = request.args.get('q2')
    answer = Answers(
        id=user.id,
        q1=q1,
        q2=q2
    )
    db.session.add(answer)
    db.session.commit()
    
    return 'Ok' # пользователь попадает на страницу, где напсано только Ок

### Как получить из базы что-то сложное?

Нам нужна какая-то статистика на сайт, чтобы любопытные могли посмотреть сколько человек уже прошли анкету и какие примерно ответы получаются

In [ ]:
from sqlalchemy import func # это позволит использовать функции типа среднее, максимум, минимум и т.д.

In [ ]:
@app.route('/stats')
def stats():
    # заводим словарь для значений (чтобы не передавать каждое в render_template)
    all_info = {}
    
    age_stats = db.session.query(
        func.avg(User.age), # средний возраст AVG(user.age)
        func.min(User.age), # минимальный возраст MIN(user.age)
        func.max(User.age)  # максимальный возраст MAX(user.age)
    ).one() # берем один результат (он всего и будет один)
    
    all_info['age_mean'] = age_stats[0]
    all_info['age_min'] = age_stats[1]
    all_info['age_max'] = age_stats[2]
    
    # это простой запрос, можно прямо у таблицы спросить
    all_info['total_count'] = User.query.count() # SELECT COUNT(age) FROM user
    
    # SELECT AVG(q1) FROM answers
    all_info['q1_mean'] = db.session.query(func.avg(Answers.q1)).one()[0]
    
    # SELECT q1 FROM answers
    q1_answers = db.session.query(Answers.q1).all()
    
    return render_template('results.html', all_info=all_info)

**Остается только адаптировать под себя и поработать над оформлением!**

## Задание

1. Прочитать условие домашнего задания
2. Придумать свою анкету
3. Добавить параметры пользователя
4. Добавить свои вопросы в базу
5. Отредактировать анкету
6. Дополнить считывание и запись новых параметров
7. Протестировать запись в базу
8. Подумать над статистикой
9. Сделать запросы в базу
10. Вывести результат